## Setup & Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pvlib
import json
import os
from pvlib.pvsystem import PVSystem, Array, FixedMount
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
import plotly.graph_objects as go

PARENT_DATA_DIR = os.getenv('PARENT_DATA_DIR')
if PARENT_DATA_DIR is None:
    raise ValueError("PARENT_DATA_DIR environment variable is not set")

## Import PRiOT data

In [28]:
deviceNames

['a001035',
 'a001036',
 'a001096',
 'a001099',
 'a001116',
 'a001131',
 'a001138',
 'a001155',
 'a001159',
 'a001163',
 'a001164',
 'a001190',
 'a001195',
 'a001204',
 'a001205',
 'a001209',
 'a001214',
 'a001230',
 'a001231',
 'a001265',
 'a001266',
 'a001286',
 'a001287',
 'a001296',
 'a001299',
 'a001301',
 'a001305',
 'a001318',
 'a001336',
 'a001359',
 'a001364',
 'a001379',
 'a001381',
 'a001383',
 'a001386',
 'a001387',
 'a001395',
 'a001403',
 'a001406',
 'a001407',
 'a001413',
 'a001417',
 'a001426',
 'a001435',
 'a001436',
 'a001454',
 'a001456',
 'a001464',
 'a001473',
 'a001477',
 'a001487',
 'a001494',
 'a001513',
 'a001539',
 'a001542',
 'a001553',
 'a001566',
 'a001584',
 'a001633',
 'a001634']

In [43]:
dataDirpath = PARENT_DATA_DIR + r"\PRiOT\dataExport_3400_daily"

# Load the metadata JSON file
metadataFilepath = os.path.join(dataDirpath, "metadata.json")

with open(metadataFilepath, 'r') as f:
    metadata = json.load(f)

# Load all csv files from the data directory
dfs = {}
for file in os.listdir(dataDirpath):
    if file.endswith(".csv"):
        deviceName = file.split("_")[0]
        dfs[deviceName] = pd.read_csv(os.path.join(dataDirpath, file))
        dfs[deviceName]['Timestamp'] = pd.to_datetime(dfs[deviceName]['Timestamp'], unit='ms', utc=True).dt.tz_convert('Europe/Zurich')
        dfs[deviceName]['energy_daily_norm'] = dfs[deviceName]['tt_forward_active_energy_total_toDay'] / metadata[deviceName]['metadata']['pv_kwp']

deviceNames = list(dfs.keys())

In [48]:
metadata['a001099']['metadata'].keys()

dict_keys(['pv_kwp', 'pv_mod2_wp', 'pv_mod1_number', 'pv_mod2_azimut', 'pv_mod1_azimut', 'pv_mod2_manufacturer', 'loc_latitude', 'pv_pv_type', 'pv_mod1_manufacturer', 'loc_longitude', 'pv_mod2_tilt', 'pv_mod2_type', 'pv_mod2_number', 'pv_mod1_tilt', 'pv_mod1_type', 'pv_mod1_wp'])

## Simulate production

In [ ]:
#  TODO ICI
for deviceName in deviceNames:
    latitude = metadata[deviceName]['metadata']['loc_latitude']
    longitude = metadata[deviceName]['metadata']['loc_longitude']
    altitude = 533

    location = Location(latitude=latitude, longitude=longitude, altitude=altitude, tz='Europe/Zurich', name=deviceName)

    array1 = Array(
            mount=FixedMount(surface_tilt=30, surface_azimuth=241, racking_model = 'open_rack'),
            module_parameters={'pdc0': 55, 'gamma_pdc' : -0.004},
            module_type = 'glass_polymer',
            modules_per_string = 445, # 445 modules for 1 string or 89 modules for 5 strings is the same
            strings = 1,
            temperature_model_parameters=pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer'],
        )

    system = PVSystem(arrays=[array1], inverter_parameters={'pdc0': 24475, 'eta_inv_nom': 0.96})

    modelChain = ModelChain(system, location ,clearsky_model = 'ineichen', aoi_model='no_loss', spectral_model="no_loss")

    times_minutes = pd.date_range(start='2022-01-01', end='2022-06-30', freq='5min', tz='UTC')
    cs_minutes_power = location.get_clearsky(times_minutes)  # ineichen with climatology table by default

    modelChain.run_model(cs_minutes_power)

## Plot result

In [44]:
# Plot the data in different trace on the same figure
fig = go.Figure()
for device, df in dfs.items():
    fig.add_trace(go.Scatter(x=df['Timestamp'], y=df['energy_daily_norm'], name=device, mode='markers', visible='legendonly'))
    # fig.add_trace(go.Bar(x=df['Timestamp'], y=df['energy_daily_norm'], name=device, width=24 * 60 * 60 * 1000 * 0.8, visible='legendonly'))


fig.update_layout(title='Superposition of Daily Data', xaxis_title='Timestamp', yaxis_title='Energy')
fig.show()